In [1]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

In [2]:
!ls PetImages/train

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!mkdir PetImages/train

In [12]:
!mv PetImages/Cat/ PetImages/train/Cat/

In [14]:
!mv PetImages/Dog/ PetImages/train/Dog/

In [17]:
!mkdir PetImages/test

In [19]:
!mkdir PetImages/test/Cat

In [20]:
!mkdir PetImages/test/Dog

## разделение данных на тестовое и обучающее множество

In [22]:
import os, shutil
import random as rn

In [21]:
train_dirs = ['PetImages/train/Dog', 
              'PetImages/train/Cat']

test_dirs =  ['PetImages/test/Dog', 
              'PetImages/test/Cat']

In [23]:
# Страшный код
for i in range(len(train_dirs)):
    fnames = os.listdir(train_dirs[i])
    toCopy = len(fnames)//5

    filesToCopy = []
    for j in range(toCopy):
        filesToCopy.append(rn.choice(fnames))
    
    # cOPY
         
    for fi in filesToCopy:
        src = os.path.join(train_dirs[i],fi)
        dst = os.path.join(test_dirs[i], fi)
        shutil.copyfile(src,dst)

    
    for fi in filesToCopy:
        try:
            src = os.path.join(train_dirs[i],fi)
            os.remove(src)
            print(src+' deleted')
            #time.sleep(1)
        except:
            pass


PetImages/train/Dog/1189.jpg deleted
PetImages/train/Dog/884.jpg deleted
PetImages/train/Dog/4559.jpg deleted
PetImages/train/Dog/6712.jpg deleted
PetImages/train/Dog/1648.jpg deleted
PetImages/train/Dog/4591.jpg deleted
PetImages/train/Dog/1464.jpg deleted
PetImages/train/Dog/8655.jpg deleted
PetImages/train/Dog/9717.jpg deleted
PetImages/train/Dog/1289.jpg deleted
PetImages/train/Dog/1414.jpg deleted
PetImages/train/Dog/8137.jpg deleted
PetImages/train/Dog/3953.jpg deleted
PetImages/train/Dog/3033.jpg deleted
PetImages/train/Dog/6617.jpg deleted
PetImages/train/Dog/2536.jpg deleted
PetImages/train/Dog/7524.jpg deleted
PetImages/train/Dog/215.jpg deleted
PetImages/train/Dog/9996.jpg deleted
PetImages/train/Dog/11824.jpg deleted
PetImages/train/Dog/11588.jpg deleted
PetImages/train/Dog/2.jpg deleted
PetImages/train/Dog/7124.jpg deleted
PetImages/train/Dog/820.jpg deleted
PetImages/train/Dog/12479.jpg deleted
PetImages/train/Dog/11184.jpg deleted
PetImages/train/Dog/6516.jpg deleted
Pet

In [ ]:
# осмотр получившихся директорий

## подготовка генераторов

In [ ]:
# Воспоминание о генераторах

In [ ]:
def int_generator ():
    i = 0
    while True:
        yield i
        i += 1
        

In [ ]:
j = 0
for item in int_generator():
    print (item)
    j += 1
    if j > 10:
        break

In [ ]:
# взрослый генератор

In [2]:
Pic_Y, Pic_X = 180, 180
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255)
                            
train_generator = train_datagen.flow_from_directory('PetImages/train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 20456 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('PetImages/test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 4542 images belonging to 2 classes.


In [20]:
# Обучение модели с 0

model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model.add(layers.Flatten())
model.add(layers.Dense(1, activation="sigmoid"))


In [22]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [23]:
# Новый интересный объект
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.h5",
        save_best_only=True,
        monitor="val_loss")]

In [24]:
history = model.fit(train_generator, epochs=30, validation_data=test_generator,
    callbacks=callbacks)

Epoch 1/30
320/320 [==============================] - ETA: 0s - loss: 0.6778 - accuracy: 0.5874

/home/sergey/anaconda3/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


320/320 [==============================] - 449s 1s/step - loss: 0.6778 - accuracy: 0.5874 - val_loss: 0.5726 - val_accuracy: 0.7188
Epoch 2/30
320/320 [==============================] - 482s 2s/step - loss: 0.5342 - accuracy: 0.7349 - val_loss: 0.4495 - val_accuracy: 0.7867
Epoch 3/30
320/320 [==============================] - 500s 2s/step - loss: 0.4441 - accuracy: 0.7959 - val_loss: 0.4375 - val_accuracy: 0.8096
Epoch 4/30
320/320 [==============================] - 522s 2s/step - loss: 0.3705 - accuracy: 0.8333 - val_loss: 0.3780 - val_accuracy: 0.8296
Epoch 5/30
 20/320 [>.............................] - ETA: 7:47 - loss: 0.3438 - accuracy: 0.8453

KeyboardInterrupt: 

## Добавляем расширение данных 

In [27]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model.add(layers.Flatten())
model.add(layers.Dense(1, activation="sigmoid"))


In [28]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [29]:
# Новый интересный объект
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="augmentation_model.h5",
        save_best_only=True,
        monitor="val_loss")]

In [30]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 25,
                                   shear_range = 0.15,
                                   zoom_range = 0.3,
                                   fill_mode = 'nearest')
                            
train_generator = train_datagen.flow_from_directory('PetImages/train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 20456 images belonging to 2 classes.


In [31]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('PetImages/test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 4542 images belonging to 2 classes.


In [32]:
history = model.fit(train_generator, epochs=5, validation_data=test_generator,
    callbacks=callbacks)

Epoch 1/5
320/320 [==============================] - 506s 2s/step - loss: 0.6827 - accuracy: 0.5746 - val_loss: 0.6281 - val_accuracy: 0.6268
Epoch 2/5
 96/320 [========>.....................] - ETA: 5:23 - loss: 0.5964 - accuracy: 0.6816

KeyboardInterrupt: 

## Перенос знаний!

In [4]:
# закгрузка сверточных слоев VGG16

conv_base = tf.keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))


2022-07-14 16:52:40.933531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-14 16:52:40.933604: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-14 16:52:40.933656: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sergey-home): /proc/driver/nvidia/version does not exist
2022-07-14 16:52:40.934386: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Заморозка сверточных слоев
conv_base.trainable = False

In [6]:
# Конструируем модель
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1, activation = 'sigmoid'))

In [7]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [8]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="VGG_FIX_model.h5",
        save_best_only=True,
        monitor="val_loss")]

In [ ]:
# фитим на тех же генераторах то есть с расширением данных
history = model.fit(train_generator, epochs=1, validation_data=test_generator, 
    callbacks=callbacks)

2022-07-14 16:53:10.409706: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 530841600 exceeds 10% of free system memory.


In [ ]:
## Fine Turning

In [ ]:
# Размораживаем 4 верхних свертки
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.h5",
        save_best_only=True,
        monitor="val_loss")]


In [ ]:
history = model.fit(train_generator, epochs=1, validation_data=test_generator, 
    callbacks=callbacks)

In [5]:
## Оценка модели на проверочном множестве.
model = tf.keras.models.load_model("convnet_from_scratch.h5")
test_loss, test_acc = model.evaluate(train_generator)
print(f"Test accuracy: {test_acc:.3f}")

2022-07-14 17:06:59.306282: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-14 17:06:59.306355: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-14 17:06:59.306408: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sergey-home): /proc/driver/nvidia/version does not exist
2022-07-14 17:06:59.306970: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-14 17:07:00.607380: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 excee

  1/320 [..............................] - ETA: 3:45 - loss: 0.3602 - accuracy: 0.9062

2022-07-14 17:07:01.352816: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


  2/320 [..............................] - ETA: 3:18 - loss: 0.3207 - accuracy: 0.8828

2022-07-14 17:07:02.060701: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


  3/320 [..............................] - ETA: 3:31 - loss: 0.3080 - accuracy: 0.8906

2022-07-14 17:07:02.633421: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


  4/320 [..............................] - ETA: 3:21 - loss: 0.2993 - accuracy: 0.8945

2022-07-14 17:07:03.432723: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


320/320 [==============================] - 231s 721ms/step - loss: 0.3440 - accuracy: 0.8485
Test accuracy: 0.848


In [16]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
import os
i=0
pa = "PetImages/train/Cat"
path = Path(pa).rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            i+=1
            print(img_p, i)
            os.remove(img_p)

PetImages/train/Cat/666.jpg 1


In [ ]:
#PetImages/train/Dog/11702.jpg 1

# задача ,создать функцию,1.получает картинку,2.выполняет прогнозы полученные тремя моделями на уроке3.организует
# мягкое голосование 4. возвращает строку с названием класса( кот или собака.)

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

def image_recognize(filename):
    Pic_Y, Pic_X = 180, 180
    img = image.load_img(filename,target_size = ( Pic_Y, Pic_X))
    arr = image.img_to_array(img)
    arr = np.expand_dims(arr,axis = 0)
    
    model1 = models.load_model("convnet_from_scratch_8846.h5")
    model2 = models.load_model("VGG_FIX_model_9073.h5") 
    model3 = models.load_model("fine_tuning_0946.h5") 
    
    
# fine_tuning_0946.h5" Например 0946 означает наша accuracy из кода выше в результатах( CW 29)
    pred1 = model1.predict(arr)
    pred2 = model2.predict(arr)
    pred3 = model3.predict(arr)
    
    sumthreemodel = (pred1 + pred2 + pred3)/3
    if sumthreemodel[0][0] >0.5:
        return "It's a Dog"
    else:
        return "It's a Cat"
    